In [1]:
import torch
from rlaopt.models import LinSys, SymmetricLinOp
from rlaopt.solvers import PCGConfig
from rlaopt.preconditioners import NystromConfig, IdentityConfig

In [2]:
torch.set_default_dtype(torch.float64)

In [3]:
device = torch.device("cuda:0")
reg = 0.0001

m = 35000
n=1000

singular_values = torch.arange(1, n + 1, device=device) ** -1.0
U = torch.randn(m, n, device=device)
U, _ = torch.linalg.qr(U)
V = torch.randn(n, n, device=device)
V, _ = torch.linalg.qr(V)
A = U @ torch.diag(singular_values) @ V.T
wStar = torch.randn(n, device=device)/(n**0.5)
b = A@wStar+0.1*torch.randn(m,device=device)/(m**(0.5))
Atr = A[0:30000,:]
Atst = A[30000:-1,:]
btr = b[0:30000]
btst = b[30000:-1]
#b = torch.randn(m, device=device)

In [4]:
linop = SymmetricLinOp(shape=(n, n), matvec=lambda x: A.T @ (A @ x))
system = LinSys(linop, b=A.T @ b, reg=reg)

In [5]:
def callback_fn(w, linsys, Atst, btst):
    res = torch.linalg.norm(linsys.b-(linsys.A @ w+linsys.reg * w))
    ntst = btst.shape[0]
    test_loss = 1/ntst*torch.linalg.norm(Atst@w-btst)**2
    return {"res": res, "test_loss": test_loss}

In [6]:
# Precond Configs
nystrom_config = NystromConfig(rank=100, rho=reg)
identity_config = IdentityConfig()

solver_config = PCGConfig(precond_config=nystrom_config, max_iters=500, atol=1e-6, rtol=1e-6, device=device)

In [7]:
w, log = system.solve(solver_name="pcg", solver_config=solver_config, w_init=torch.zeros(n, device=device), callback_fn=callback_fn, callback_kwargs={"Atst": Atst, "btst": btst})

(1000, 1000)


In [8]:
A.shape

torch.Size([35000, 1000])

In [9]:
log

{0: {'abs_res': tensor(0.0023, device='cuda:0'),
  'rel_res': tensor(0.1082, device='cuda:0'),
  'callback': {'res': tensor(0.0023, device='cuda:0'),
   'test_loss': tensor(2.7737e-07, device='cuda:0')}},
 10: {'abs_res': tensor(9.2209e-10, device='cuda:0'),
  'rel_res': tensor(4.4188e-08, device='cuda:0'),
  'callback': {'res': tensor(9.2209e-10, device='cuda:0'),
   'test_loss': tensor(2.7663e-07, device='cuda:0')}}}